___

# M915 - Συστήματα Κατανόησης και Παραγωγής Κειμένου 
___
### Kylafi Christina-Theano <br><br> LT1200012

In [2]:
# imports
# essentials
import os
import random
import numpy as np
from numpy import mean, std
import math
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import json
from collections import Counter
import re, string, unicodedata
import pickle 
from datetime import datetime
import pytz
from itertools import cycle
from scipy import interp 
import time
import copy
import json
import csv
from ast import literal_eval

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
from torch.utils.data import DataLoader, TensorDataset, Dataset

# SKLEARN
import sklearn
import sklearn.metrics
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import normalize, OneHotEncoder, label_binarize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, precision_score, recall_score, log_loss, plot_confusion_matrix, roc_curve, auc, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn import utils
from sklearn.svm import SVC

# NLTK
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer



# BERT
# !pip install transformers
# !pip install pytorch-pretrained-bert

import transformers
from transformers.data.processors.squad import SquadV2Processor
from transformers.data import squad_convert_examples_to_features
from transformers import AutoTokenizer
from transformers import BertTokenizer, BertModel, BertForPreTraining, BertTokenizerFast, AdamW, BertForQuestionAnswering, DistilBertTokenizer, DistilBertForQuestionAnswering, DistilBertTokenizerFast, DistilBertModel
from torch.utils.data import DataLoader
from transformers import AdamW
# from tqdm import tqdm_notebook as tqdm
from tqdm import tqdm

import warnings

# MORE INSTALLATIONS & IMPORTS
# !pip install yellowbrick
# !pip install advertools
# !pip install vaderSentiment
# !pip install ekphrasis
# !pip install tweet-preprocessor

from wordcloud import WordCloud
# import advertools as adv
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# from ekphrasis.classes.segmenter import Segmenter
# import preprocessor as p
import multiprocessing
from shutil import copy
import seaborn as sns
from gensim.models import Word2Vec
from IPython.display import Image
from ast import literal_eval
from tqdm.notebook import tqdm
from IPython.display import FileLink

print("\nImports Done !\n")

# Device settings
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f"Working on {device}")

In [11]:
# Functions
def squad_df(file_path, record_path=['data','paragraphs','qas','answers']):
    file = json.loads(open(file_path).read())
    
    # parsing different level's in the json file
    js = pd.json_normalize(file, record_path)
    m = pd.json_normalize(file, record_path[:-1])
    r = pd.json_normalize(file,record_path[:-2])
    
    # combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    m['context'] = idx
    data = m[['id','context','question','answers']].set_index('id').reset_index()
    data['context_id'] = data['context'].factorize()[0]
    return data


# Add answer end index
def add_ans_ind(qna_df):
    texts = qna_df["context"]
    answers = qna_df["answers"]
    for row,(text,answers) in enumerate(zip(texts,answers)):
        for a_num,a in enumerate(answers):
            a_text = a["text"]
            a_start = a["answer_start"]
            a_end = a_start
            
            a_end = int(a_start+len(a_text))
            if text[a_start:a_end] == a_text:
                qna_df.loc[row,"answers"][a_num]["answer_end"]=a_end
            else:
                if a_start==0:
                    continue
                else:
                    if a_start==1:
                        if text[a_start-1:a_end-1] == a_text:
                            qna_df.loc[row,"answers"][a_num]["answer_start"]= a_start - 1
                            qna_df.loc[row,"answers"][a_num]["answer_end"]= a_end - 1

                    else:
                        if text[a_start-2:a_end-2] == a_text:
                            qna_df.loc[row,"answers"][a_num]["answer_start"]= a_start - 2
                            qna_df.loc[row,"answers"][a_num]["answer_end"]= a_end - 2

           
    return qna_df


def data_stats(df, type="Train"):
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', return_token_type_ids=True)
    enc = 

# pr_list = add_ans_ind(qna_train_df_quac)
# pr_list = add_ans_ind(qna_dev_df_quac)

In [13]:
# directories' paths
squad_train_path = "/kaggle/input/bert-code/bert/train-v1.1.json"
squad_dev_path = "/kaggle/input/bert-code/bert/dev-v1.1.json"
train_df = squad_df(squad_train_path)
dev_df = squad_df(squad_dev_path)

results_path = "/kaggle/working/results"
if not os.path.exists(results_path):
    os.makedirs(results_path)
    
eval_v1_path = "/kaggle/input/bert-code/bert/evaluate.py"
eval_v2_path = "/kaggle/input/httpsgithubcomsomiltgbert/evaluate-v2.0.py"

In [5]:
# clean GPU cache
import gc
import torch
def clean_GPU_cache(print_sum=False):
  if print_sum:  
    print("\nBefore\n")
    print(torch.cuda.memory_summary(device=device, abbreviated=False))
    torch.cuda.memory_summary(device=None, abbreviated=False)
  torch.cuda.empty_cache()
  gc.collect()
  torch.cuda.empty_cache()
  if print_sum:  
    print("\n\nAfter\n")
    print(torch.cuda.memory_summary(device=device, abbreviated=False))

def check_gpu():
  clean_GPU_cache()
  print("\n")
  !nvidia-smi
  print("\n")

if device=="cuda":
    check_gpu()
# clean_GPU_cache()

In [9]:
# load distilBERT tokenizer & model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', return_token_type_ids=True)
# tokenizer.model_max_length = 250

model = DistilBertModel.from_pretrained("distilbert-base-uncased")


In [14]:
# data processing
# add end offset to the answers dictionaries
train_df=add_ans_ind(train_df)
dev_df=add_ans_ind(dev_df)

# tokenize
train_encodings = tokenizer(list(train_df['context']), list(train_df['question']), max_length=tokenizer.model_max_length, truncation="only_first", padding='max_length')
dev_encodings = tokenizer(list(dev_df['context']), list(dev_df['question']), max_length=tokenizer.model_max_length, truncation="only_first", padding='max_length')


In [ ]:
type(dev_encodings)

In [ ]:
cols= [ c for c in train_encodings.columns ]
values= [ train_encodings[c] for c in cols ]
train_enc_df=pd.DataFrame(values,columns=cols)

cols= [ c for c in dev_encodings.columns ]
values= [ dev_encodings[c] for c in cols ]
dev_enc_df=pd.DataFrame(values,columns=cols)

train_enc_df.to_csv(os.path.join(results_path,"SQuADv1_train_encodings.csv"), index=False, header=True)
dev_enc_df.to_csv(os.path.join(results_path,"SQuADv1_dev_encodings.csv"), index=False, header=True)

In [ ]:
# model training
if device=="cuda":
    check_gpu()

## 📚 References

1. [text]()
2. [text]()
3. [text]()
4. [text]()
5. [text]()
6. [NLP - Document Retrieval for Question Answering](https://www.kaggle.com/code/leomauro/nlp-document-retrieval-for-question-answering)
7. [text]() 
8. [text]()
9. [text]()
10. [text]()
11. [text]()
12. [text]()
13. [text]()
14. [text]()
15. [text]()
16. [text]()
17. [text]()



In [11]:
# eval_set = "NewsQA"
# preds_file = f"/content/Results/preds_ftune{ftune_set}_eval{eval_set}.json"

# total_preds = total_preds_list(model,dev_dataset,device,dev_df)

# with open(preds_file, "w") as outfile:
#     json.dump(ast.literal_eval(json.dumps(total_preds)), outfile)
# !python3 {eval_file} ../input/squad2/dev-v2.0.json preds.json {preds_file} > ./ftune{ftune_set}_eval{eval_set}.txt
# !python3 {eval_file} ../input/squad2/dev-v2.0.json preds.json {preds_file}